In [ ]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt

from gcspy.graph_of_convex_sets import GraphOfConvexSets
from gcspy.shortest_path_problem import ShortestPathProblem

In [ ]:
# Initialize graph.
gcs = GraphOfConvexSets()

# Add vertices.
u = gcs.add_vertex('u')
v = gcs.add_vertex('v')
w = gcs.add_vertex('w')

# Add continuous variables.
xu = u.add_variable(2)
xv = v.add_variable(2)
xw = w.add_variable(2)

# Constraints on the vertices.
u.add_constraint(xu <= 0)
u.add_constraint(xu >= -.1)
v.add_constraint(xv >= .4)
v.add_constraint(xv <= .6)
w.add_constraint(xw == 1)

# Add edges.
e = gcs.add_edge(u, v)
f = gcs.add_edge(u, w)
g = gcs.add_edge(v, w)
h = gcs.add_edge(w, v)

# Costs on the edges.
P = np.eye(2)
e.add_length(cp.quad_form(xv - xu, P))
f.add_length(cp.quad_form(xw - xu, P))
g.add_length(cp.quad_form(xw - xv, P))
h.add_length(cp.norm(xv - xw, 2))

In [ ]:
gcs.graphviz()

In [ ]:
spp = ShortestPathProblem(gcs)
sol = spp.solve(u, w, relaxation=1)

In [ ]:
plt.figure()
for v in gcs.vertices:
    v.plot(sol.x[v])
for e in gcs.edges:
    color = 'b' if np.isclose(sol.y[e], 1) else 'k'
    e.plot(sol.x[e.u], sol.x[e.v], color=color)